# Search
## Ghazal Kalhor

<div style="text-align: justify; font-weight: bold;">
<i>Abstract</i> — In this computer assignment, we want to find a
suitable solution for “Snake and Foods” problem; this is
done by using the Uninformed Search and Informed Search
algorithms such as BFS, IDS, and A* that we learnt in Artificial
Intelligence.  
    
<i>Keywords</i> — Uninformed Search, Informed Search, BFS, IDS,
A*, Artificial Intelligence
 </div>

### Introduction
<br/>
<div style="text-align: justify;"> 
The goal of this computer assignment to help our agent to
do its job. Our agent is a snake that has to eat all the
foods in the map with minimum possible actions.
</div>

### Importing Libraries
<br/>
<div style="text-align: justify;">
In this part, some of the necessary libraries were imported in order to use their helpful functions.
</div>

In [1]:
import math
import heapq
from time import time

### Defining Constants
<br/>
<div style="text-align: justify;">
In this part constant values are defined in order to make the code more readable and more flexible to change.
</div>

In [2]:
HEAD = 0

SUCCESS = True
FAILURE = False

FOODS_X = 0
FOODS_Y = 1
SCORES = 2
HEAD_X = 3
HEAD_Y = 4
WIDTH = 5
HEIGHT = 6

LEFT = 'L'
RIGHT = 'R'
UP = 'U'
DOWN = 'D'

FAST_HEURISTIC = 0
SLOW_HEURISTIC = 1
HIGH_WEIGHTED = 2
LOW_WEIGHTED = 3

SMALL_ALPHA = 1.5
BIG_ALPHA = 5

TESTCASE1 = "test1.txt"
TESTCASE2 = "test2.txt"
TESTCASE3 = "test3.txt"

NEWLINE = '\n'

### Modeling the Problem
<br/>
<div style="text-align: justify;">
In this problem we considered each <b>state</b> consisting of a
list of the positions of the foods and their scores, and a list of
the position of the snake. The <b>initial state</b> is read from the file. The <b>goal state</b> is obtained,
when the list of foods is empty. It means that the agent has
done its job. Each <b>action</b> can be moving the object one unit to
the left, right, top and bottom. If after an action the agent meets a food, it must eat it on that action. We defined the <b>Node</b> class for this goal.
</div>

In [3]:
class Node:
    def __init__(self, foods_x, foods_y, scores, snake_x, snake_y, width, height,\
                 action = None, increase = False, parent = None):
        self.foods_x = foods_x
        self.foods_y = foods_y
        self.scores = scores
        self.snake_x = snake_x
        self.snake_y = snake_y
        self.width = width
        self.height = height
        self.parent = parent
        self.action = action
        self.path_cost = 0
        self.increase = increase

        if self.parent:
            self.path_cost = parent.get_cost() + 1
            
    def get_cost(self):
        return self.path_cost
    
    def contains_goal_state(self):
        food_count = len(self.foods_x)
        
        if food_count == 0:
            return True
        
        return False
    
    def has_length_one(self):
        if len(self.snake_x) == 1:
            return True
        
        return False
    
    
    def get_children(self):
        children = set()
        self.add_top_child(children)
        self.add_right_child(children)
        self.add_bottom_child(children)
        self.add_left_child(children)
        
        return children
    
    def add_top_child(self, children):
        i = self.snake_x[HEAD]
        j = self.snake_y[HEAD]
        foods_x = self.foods_x[:]
        foods_y = self.foods_y[:]
        scores = self.scores[:]
        snake_x = self.snake_x[:]
        snake_y = self.snake_y[:]
        width = self.width
        height = self.height
        increase = False
        
        if i == 0:
            i = (height - 1)        
        else:
            i -= 1
                
        if self.has_length_one() == False:
            if snake_x[HEAD+1] == i and snake_y[HEAD+1] == j:
                return 
            for index in range(len(snake_x)):
                if snake_x[index] == i and snake_y[index] == j:
                    if index == len(snake_x)-1 and self.increase == False:
                        break
                    else:
                        return
            
        snake_x.insert(HEAD, i)
        snake_y.insert(HEAD, j)
        
        if self.increase == False:     
            snake_x.pop()
            snake_y.pop()
        
        l = len(foods_x)
        for index in range(l):
            if foods_x[index] == i and foods_y[index] == j:
                increase = True
                if scores[index] == 1:
                    foods_x.pop(index)
                    foods_y.pop(index)
                    scores.pop(index)
                else:
                    scores[index] = 1
                
                break
            
        top_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height, UP, increase, self)
        children.add(top_node)
        
    def add_bottom_child(self, children):
        i = self.snake_x[HEAD]
        j = self.snake_y[HEAD]
        foods_x = self.foods_x[:]
        foods_y = self.foods_y[:]
        scores = self.scores[:]
        snake_x = self.snake_x[:]
        snake_y = self.snake_y[:]
        width = self.width
        height = self.height
        increase = False
        
        if i == (height - 1):
            i = 0     
        else:
            i += 1
            
        if self.has_length_one() == False:
            if snake_x[HEAD+1] == i and snake_y[HEAD+1] == j:
                return
            for index in range(len(snake_x)):
                if snake_x[index] == i and snake_y[index] == j:
                    if index == len(snake_x)-1 and self.increase == False:
                        break
                    else:
                        return
            
        snake_x.insert(HEAD, i)
        snake_y.insert(HEAD, j)
        
        if self.increase == False:     
            snake_x.pop()
            snake_y.pop()
        
        l = len(foods_x)
        for index in range(l):
            if foods_x[index] == i and foods_y[index] == j:
                increase = True
                if scores[index] == 1:
                    foods_x.pop(index)
                    foods_y.pop(index)
                    scores.pop(index)
                else:
                    scores[index] = 1
                
                break
            
        bottom_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height, DOWN, increase, self)
        children.add(bottom_node)
        
    def add_right_child(self, children):
        i = self.snake_x[HEAD]
        j = self.snake_y[HEAD]
        foods_x = self.foods_x[:]
        foods_y = self.foods_y[:]
        scores = self.scores[:]
        snake_x = self.snake_x[:]
        snake_y = self.snake_y[:]
        width = self.width
        height = self.height
        increase = False
        
        if j == (width - 1):
            j = 0   
        else:
            j += 1
            
        if self.has_length_one() == False:
            if snake_x[HEAD+1] == i and snake_y[HEAD+1] == j:
                return
            for index in range(len(snake_x)):
                if snake_x[index] == i and snake_y[index] == j:
                    if index == len(snake_x)-1 and self.increase == False:
                        break
                    else:
                        return
            
        snake_x.insert(HEAD, i)
        snake_y.insert(HEAD, j)
        
        if self.increase == False:     
            snake_x.pop()
            snake_y.pop()
        
        l = len(foods_x)
        
        for index in range(l):
            if foods_x[index] == i and foods_y[index] == j:
                increase = True
                if scores[index] == 1:
                    foods_x.pop(index)
                    foods_y.pop(index)
                    scores.pop(index)
                else:
                    scores[index] = 1
                
                break
            
        right_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height, RIGHT, increase, self)
        children.add(right_node)     
        
    
    def add_left_child(self, children):
        i = self.snake_x[HEAD]
        j = self.snake_y[HEAD]
        foods_x = self.foods_x[:]
        foods_y = self.foods_y[:]
        scores = self.scores[:]
        snake_x = self.snake_x[:]
        snake_y = self.snake_y[:]
        width = self.width
        height = self.height
        increase = False
        
        if j == 0:
            j = (width - 1)        
        else:
            j -= 1
            
        if self.has_length_one() == False:
            if snake_x[HEAD+1] == i and snake_y[HEAD+1] == j:
                return
            for index in range(len(snake_x)):
                if snake_x[index] == i and snake_y[index] == j:
                    if index == len(snake_x)-1 and self.increase == False:
                        break
                    else:
                        return
            
        snake_x.insert(HEAD, i)
        snake_y.insert(HEAD, j)
        
        if self.increase == False:     
            snake_x.pop()
            snake_y.pop()
        
        l = len(foods_x)
        for index in range(l):
            if foods_x[index] == i and foods_y[index] == j:
                increase = True
                if scores[index] == 1:
                    foods_x.pop(index)
                    foods_y.pop(index)
                    scores.pop(index)
                else:
                    scores[index] = 1
                
                break
            
        left_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height, LEFT, increase, self)
        children.add(left_node)
        
    def get_string(self):
        l = ''
        for i in range(len(self.snake_x)):
            l += str(self.snake_x[i])
            l += '.'
            l += str(self.snake_y[i])
            l += '.'
            
        for i in range(len(self.foods_x)):
            l += str(self.foods_x[i])
            l += '.'
            l += str(self.foods_y[i])
            l += '.'
            l += str(self.scores[i])
            l += '.'
    
        return l
    
    def get_state(self):
        info = list()
        info.append(self.foods_x)
        info.append(self.foods_y)
        info.append(self.scores)
        info.append(self.snake_x[HEAD])
        info.append(self.snake_y[HEAD])
        info.append(self.width)
        info.append(self.height)
        return info

### Defining FIFO Queue
<br/>
<div style="text-align: justify;">
FIFO is an abbreviation for first in, first out. It is a method for handling data structures where the first element is processed first and the newest element is processed last.
</div>

In [4]:
class FIFO_queue:
    def __init__(self):
        self.dataset = []

    def enqueue(self, data):
        self.dataset.append(data)

    def dequeue(self):
        if self.is_empty() == True:
            return
        
        data = self.dataset[0]
        self.dataset.pop(0)

        return data

    def initialize(self, data):
        self.enqueue(data)

    def is_empty(self):
        if len(self.dataset) == 0:
            return True

### BFS Algorithm
<br/>
<div style="text-align: justify;">
In this section, we used the BFS algorithm to find a suitable
solution for the problem. For this goal, we defined the class BFS
that has a frontier set that is implemented by FIFO queue to
store the list of unexpanded nodes, an explored set that every
time we expand a node, we add its state to this set, Also we
stored number of visited states and number of unique visited
states in this class that we update them during the execution of
the algorithm. The algorithm runs when we call the “play”
function on its instance. At first we initialize the frontier set by
the “initial_node”, then a loop continues until we did not reach
a goal node or the frontier set is non-empty. At each step we
enqueue a node from the frontier set. If it contains the goal state,
we return the solution, else we expand it by adding its children
to the frontier set.
</div>

In [5]:
class BFS:
    def __init__(self, foods_x, foods_y, scores, snake_x, snake_y, width, height):
        self.number_of_unique_states = 0
        self.number_of_states = 0
        self.path_cost = 0
        self.inital_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height)
        self.frontier = FIFO_queue()
        self.explored = set()
        self.unique_states = set()
        self.goal_node = None
        self.path = ''
 
    def play(self):
        node = self.inital_node

        if node.contains_goal_state():
            self.path_cost = node.path_cost
            self.goal_node = node
            return SUCCESS

        self.frontier.initialize(node)

        while not self.frontier.is_empty():
            node = self.frontier.dequeue()

            self.number_of_states += 1
            if node.get_string() in self.explored:
                continue

            self.explored.add(node.get_string())

            if not node.get_string() in self.unique_states:
                self.unique_states.add(node.get_string())
                self.number_of_unique_states += 1

            if node.contains_goal_state():
                self.path_cost = node.path_cost
                self.goal_node = node
                return SUCCESS
            
            for child in node.get_children():
                if not child.get_string() in self.explored:
                    if child.contains_goal_state():
                        self.path_cost= child.path_cost
                        self.goal_node = child
                        return SUCCESS

                    self.frontier.enqueue(child)
                    
        return FAILURE

    def show_result(self):
        node = self.goal_node
        
        while node.parent != None:
            self.path += node.action
            node = node.parent
        
        self.path = ''.join(reversed(self.path))
            
        print("***Breadth First Search***")
        print("Path Cost: " + str(self.path_cost))
        print("Path: " + self.path)
        print("Number of Visited States: " + str(self.number_of_states))
        print("Number of Unique Visited States: " + str(self.number_of_unique_states))

### Defining LIFO Queue
<br/>
<div style="text-align: justify;">
LIFO is an abbreviation for last in, first out. It is a method for handling data structures where the first element is processed last and the last element is processed first.
</div>

In [6]:
class LIFO_queue:
    def __init__(self):
        self.dataset = []

    def enqueue(self, data):
        self.dataset.append(data)

    def dequeue(self):
        if self.is_empty() == True:
            return
        data = self.dataset[len(self.dataset)-1]

        self.dataset.pop(len(self.dataset)-1)

        return data

    def initialize(self, data):
        self.enqueue(data)

    def is_empty(self):
        if len(self.dataset) == 0:
            return True
        return False

### IDS Algorithm
<br/>
<div style="text-align: justify;">
In this section, we used the IDS algorithm to find a suitable
solution for the problem. For this goal, we defined the class IDS
that has a frontier set that is implemented by LIFO queue to
store the list of unexpanded nodes, an explored set that every
time we expand a node, we add its state to this set and a
dictionary that has each state as its key and minimum depth that
this state has been visited at. Also we stored number of visited
states and number of unique visited states in this class that we
update them during the execution of the algorithm. The
algorithm runs when we call the “play” function on its
instance. This function calls the subroutine “dls” that executes
a depth limited search with the specified depth. We used a
recursive approach for implementing this function. At first we
check that the given node contains the goal state or not, if yes
we return a solution, else we call the function on its children if
we did non visit its state or we visited it at the lower depth. The
“play” function call its subroutine in a loop by incrementing the
depth, until it reaches the success.
</div>

In [7]:
class IDS:
    def __init__(self, foods_x, foods_y, scores, snake_x, snake_y, width, height):
        self.number_of_unique_states = 0
        self.number_of_states = 0
        self.path_cost = 0
        self.inital_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height)
        self.frontier = LIFO_queue()
        self.explored = set()
        self.unique_states = set()
        self.goal_node = None
        self.path = ''
        self.min_depth_explored = {}


    def dls(self, node, depth):
        self.number_of_states += 1

        if not node.get_string() in self.unique_states:
            self.unique_states.add(node.get_string())
            self.number_of_unique_states += 1

        if node.contains_goal_state():
            self.goal_node = node
            return SUCCESS

        if depth <= 0:
            return FAILURE

        self.min_depth_explored[node.get_string()] = depth

        for child in node.get_children():
            if (not child.get_string() in self.min_depth_explored) or \
            self.min_depth_explored[child.get_string()] < depth-1:

                if self.dls(child, depth-1) == SUCCESS:
                    return SUCCESS
        return FAILURE

    def play(self):
        node = self.inital_node
        self.min_depth_explored = {}
        depth = 1
        while True:
            if self.dls(node, depth) == SUCCESS:
                self.path_cost = depth
                return SUCCESS
            depth +=1
        return FAILURE



    def show_result(self):
        node = self.goal_node
        
        while node.parent != None:
            self.path += node.action
            node = node.parent
            
        print("***Iterative Deepening Search***")
        print("Path Cost: " + str(self.path_cost))
        print("Path: " + self.path)
        print("Number of Visited States: " + str(self.number_of_states))
        print("Number of Unique Visited States: " + str(self.number_of_unique_states))

### Defining Priority Queue
<br/>
<div style="text-align: justify;">
A priority queue is an abstract data type similar to a regular queue or stack data structure in which each element additionally has a <b>priority</b> associated with it. In a priority queue, an element with high priority is served before an element with low priority. 
</div>

In [8]:
class HEAP_queue:
    def __init__(self):
        self.dataset = []
        self.top = 0

    def enqueue(self, data, criteria):
        heapq.heappush(self.dataset, (criteria, self.top, data))
        self.top += 1

    def dequeue(self):
        return heapq.heappop(self.dataset)[-1]

    def initialize(self, data, criteria):
        self.enqueue(data, criteria)

    def is_empty(self):
        if len(self.dataset) == 0:
            return True
        return False

### A* Algorithm
<br/>
<div style="text-align: justify;">
In this section, we used the A* algorithm to find a suitable
solution for the problem. For this goal, we defined the class A*
that has a frontier set that is implemented by min-heap queue
that its criteria for choosing a node is the value of the evaluating
function “f”. The argument of this function is a node and the
return value of this function is sum of the path_cost of the node
and the value of the heuristic function. We used two heuristic
for implementing this function. 

The <b>first heuristic</b> is to calculate
sum of the scores of the remaning foods in the map. This
heuristic is admissible and consistent.     

<b>Proof of Amissiblity:</b> This heuristic never overestimates the cost to reach the goal, because at each state, at least one action must be done in order to eat a food and reduce its score by one. So the agent needs to do at least <i>s</i> actions in order to reach the goal state where <i>s</i> equals to sum of the scores of the remaning foods in the map.

$$ h(n) \leqslant \text{h*}(n) $$<br>    

<b>Proof of Consistency:</b> In this heuristic, for
every node <i>a</i> and every successor <i>b</i> of <i>a</i> generated by any action, the estimated cost of
reaching the goal from <i>a</i> is no greater than the step cost of getting to <i>b</i> plus the estimated
cost of reaching the goal from <i>b</i>.

$$ h(a) \leqslant cost(a, b) + h(b) $$<br>  

The condition is satisfied because when the agent moves from a to b, it has eaten 0  or 1 food. We can assume that h(a) is k and the agent has eaten s number of foods. So based on this assumption, h(b) is k-s. 

Moreover, we know that cost(a, b) is more than or equal to s.
                      
$$ h(a) = k, $$ $$ h(b) = k-s, $$ $$ s\leqslant cost(a, b) $$<br>
                        $$ k \leqslant cost(a, b) + k-s $$<br>

$$ h(a) \leqslant cost(a, b) + h(b) $$<br>                          

The <b>second heuristic</b> is to calculate
sum of the maximum horizontal distance and the maximum vertical distance of the snake's head from the foods. This
heuristic is admissible.

<b>Proof of Amissiblity:</b> This heuristic never overestimates the cost to reach the goal, because at each state, snake's head must do at least this number of actions in order to reach the goal state.

$$ h(n) \leqslant \text{h*}(n) $$<br>    

A* is much faster than the previous algorithms. It has an explored set that every time we expand a node, we add its state to this set, Also we stored number of visited states and number of unique visited states in this class that we update them during the execution of the algorithm. The algorithm runs when we call the “play” function on its instance. At first we initialize the frontier set by the “initial_node”, then a loop continues until we did not reach the goal state or the frontier set is non-empty. At each step we enqueue a node from the frontier set. If it contains the goal state, we return the solution, else we expand it by adding its children to the frontier set.
</div>


In [9]:
class A_STAR:
    def __init__(self, foods_x, foods_y, scores, snake_x, snake_y, width, height, version):
        self.number_of_unique_states = 0
        self.number_of_states = 0
        self.path_cost = 0
        self.inital_node = Node(foods_x, foods_y, scores, snake_x, snake_y, width, height)
        self.frontier = HEAP_queue()
        self.explored = set()
        self.unique_states = set()
        self.goal_node = None
        self.path = ''
        self.version = version

    def f(self, node):
        if self.version == FAST_HEURISTIC:
            return self.g(node) + self.h_fast(node)
        elif self.version == SLOW_HEURISTIC:
            return self.g(node) + self.h_slow(node)
        elif self.version == HIGH_WEIGHTED:
            return self.g(node) + BIG_ALPHA * self.h_slow(node)
        elif self.version == LOW_WEIGHTED:
            return self.g(node) + SMALL_ALPHA * self.h_slow(node)

    def g(self, node):
        return node.path_cost

    def h_slow(self, node):
        scores = node.get_state()[SCORES]
        sum_foods = 0
        
        for i in range(len(scores)):
            sum_foods += scores[i]

        return sum_foods


    def h_fast(self, node):
        foods_x = node.get_state()[FOODS_X]
        foods_y = node.get_state()[FOODS_Y]
        head_x = node.get_state()[HEAD_X]
        head_y = node.get_state()[HEAD_Y]
        width = node.get_state()[WIDTH]
        height = node.get_state()[HEIGHT]

        max_dist_x = 0
        max_dist_y = 0

        for i in range(len(foods_x)):            
            dist_x = abs(foods_x[i]-head_x)
            dist_x = min(dist_x, abs(dist_x-width+1))
            max_dist_x = max(dist_x, max_dist_x)
            
            dist_y = abs(foods_y[i]-head_y)
            dist_y = min(dist_y, abs(dist_y-height+1))
            max_dist_y = max(dist_y, max_dist_y)
            
        max_distance = max_dist_y + max_dist_x

        return max_distance


    def play(self):
        node = self.inital_node

        if node.contains_goal_state():
            self.path_cost= node.path_cost
            self.goal_node = node
            return SUCCESS

        self.frontier.initialize(node, self.f(node))

        while not self.frontier.is_empty():
            node = self.frontier.dequeue()

            self.number_of_states += 1
            if node.get_string() in self.explored:
                continue

            self.explored.add(node.get_string())

            if not node.get_string() in self.unique_states:
                self.unique_states.add(node.get_string())
                self.number_of_unique_states += 1

            if node.contains_goal_state():
                self.path_cost= node.path_cost
                self.goal_node = node
                return SUCCESS

            for child in node.get_children():
                if not child.get_string() in self.explored:
                    if child.contains_goal_state():
                        self.path_cost= child.path_cost
                        self.goal_node = child
                        return SUCCESS
                    
                    self.frontier.enqueue(child, self.f(child))

        return FAILURE
    
    def show_result(self):
        node = self.goal_node
        
        while node.parent != None:
            self.path += node.action
            node = node.parent
        
        self.path = ''.join(reversed(self.path))
        
        if self.version == FAST_HEURISTIC:
            print("***Fast A_STAR Search***")          
        elif self.version == SLOW_HEURISTIC:
            print("***Slow A_STAR Search***")   
        elif self.version == HIGH_WEIGHTED:
            print("***Weighted A_STAR Search with Alpha = " + str(BIG_ALPHA) + "***")   
        elif self.version == LOW_WEIGHTED:
            print("***Weighted A_STAR Search with Alpha = " + str(SMALL_ALPHA) + "***")  
        
            
        print("Path Cost: " + str(self.path_cost))
        print("Path: " + self.path)
        print("Number of Visited States: " + str(self.number_of_states))
        print("Number of Unique Visited States: " + str(self.number_of_unique_states))

### Importing Data
<br/>
<div style="text-align: justify;">
In this part, the inputs read from the file and splited and stored in lists and variables. 
</div>

In [10]:
def read_data(filename):
    foods_x = []
    foods_y = []
    scores= [] 
    snake_x = []
    snake_y = []
    width = 0
    height = 0
    with open(filename) as f:
        height, width = [int(x) for x in next(f).split(',')]
    
        head_x, head_y = [int(x) for x in next(f).split(',')]
        snake_x.append(head_x)
        snake_y.append(head_y)
    
        food_count = int(next(f))
    
        for cursor in range(food_count):
            food_x, food_y, score = [int(x) for x in next(f).split(',')] 
            foods_x.append(food_x)
            foods_y.append(food_y)
            scores.append(score)
            
    return foods_x, foods_y, scores, snake_x, snake_y, width, height

### Running BFS

In [11]:
def run_BFS(test_file):
    foods_x, foods_y, scores, snake_x, snake_y, width, height = read_data(test_file)
    bfs = BFS(foods_x, foods_y, scores, snake_x, snake_y, width, height)
    start = time()
    result = bfs.play()
    finish = time()
    bfs.show_result()
    duration = finish - start
    print("Execution Time: " + str(duration) + NEWLINE)

### Running IDS

In [12]:
def run_IDS(test_file):   
    foods_x, foods_y, scores, snake_x, snake_y, width, height = read_data(test_file)
    ids = IDS(foods_x, foods_y, scores, snake_x, snake_y, width, height)
    start = time()
    result = ids.play()
    finish = time()
    ids.show_result()
    duration = finish - start
    print("Execution Time: " + str(duration) + NEWLINE)

### Running A*

In [13]:
def run_A_STAR(test_file, version):
    foods_x, foods_y, scores, snake_x, snake_y, width, height = read_data(test_file)
    a_star = A_STAR(foods_x, foods_y, scores, snake_x, snake_y, width, height, version)
    start = time()
    result = a_star.play()
    finish = time()
    a_star.show_result()
    duration = finish - start
    print("Execution Time: " + str(duration) + NEWLINE)

### Running Testcases

In [14]:
def run_test(test_file):
    run_BFS(test_file)
    run_IDS(test_file)
    run_A_STAR(test_file, FAST_HEURISTIC)
    run_A_STAR(test_file, SLOW_HEURISTIC)
    run_A_STAR(test_file, HIGH_WEIGHTED)
    run_A_STAR(test_file, LOW_WEIGHTED)

In [15]:
print("Running Testcase #1...." + NEWLINE)
run_test(TESTCASE1)

Running Testcase #1....

***Breadth First Search***
Path Cost: 12
Path: LDRULDLUUULL
Number of Visited States: 4528
Number of Unique Visited States: 3144
Execution Time: 0.3606839179992676

***Iterative Deepening Search***
Path Cost: 12
Path: LULUULURDLDL
Number of Visited States: 13510
Number of Unique Visited States: 4502
Execution Time: 0.968317985534668

***Fast A_STAR Search***
Path Cost: 12
Path: LDULULULLUUL
Number of Visited States: 1286
Number of Unique Visited States: 1018
Execution Time: 0.4016258716583252

***Slow A_STAR Search***
Path Cost: 12
Path: LDULULULLLUU
Number of Visited States: 1991
Number of Unique Visited States: 1609
Execution Time: 0.4775385856628418

***Weighted A_STAR Search with Alpha = 5***
Path Cost: 12
Path: LDULULULLLUU
Number of Visited States: 37
Number of Unique Visited States: 37
Execution Time: 0.004992961883544922

***Weighted A_STAR Search with Alpha = 1.5***
Path Cost: 12
Path: LDLUUULLLUUL
Number of Visited States: 890
Number of Unique Visited

In [16]:
print("Running Testcase #2...." + NEWLINE)
run_test(TESTCASE2)

Running Testcase #2....

***Breadth First Search***
Path Cost: 15
Path: URDLLUUULULLLUL
Number of Visited States: 53601
Number of Unique Visited States: 37571
Execution Time: 5.0296947956085205

***Iterative Deepening Search***
Path Cost: 15
Path: LLLLUUUULLLURRL
Number of Visited States: 167514
Number of Unique Visited States: 45610
Execution Time: 10.547243118286133

***Fast A_STAR Search***
Path Cost: 15
Path: RLLURUUULLLLULL
Number of Visited States: 3563
Number of Unique Visited States: 2835
Execution Time: 0.4212965965270996

***Slow A_STAR Search***
Path Cost: 15
Path: RULLDLUUUULULLL
Number of Visited States: 33947
Number of Unique Visited States: 24492
Execution Time: 2.7613983154296875

***Weighted A_STAR Search with Alpha = 5***
Path Cost: 17
Path: ULDRRULLLUUULULLL
Number of Visited States: 1237
Number of Unique Visited States: 1145
Execution Time: 0.13314199447631836

***Weighted A_STAR Search with Alpha = 1.5***
Path Cost: 15
Path: RULLDLUUUULULLL
Number of Visited States

In [17]:
print("Running Testcase #3...." + NEWLINE)
run_test(TESTCASE3)

Running Testcase #3....

***Breadth First Search***
Path Cost: 25
Path: URRDDDDRRDRDRRURRDLLLLULU
Number of Visited States: 213889
Number of Unique Visited States: 144068
Execution Time: 18.566972017288208

***Iterative Deepening Search***
Path Cost: 25
Path: LLUULLLDRRURRRDDRRDRDDDUR
Number of Visited States: 935479
Number of Unique Visited States: 212062
Execution Time: 68.42447400093079

***Fast A_STAR Search***
Path Cost: 25
Path: RUDDDRDRRDRRRDRRULLDLLLUU
Number of Visited States: 42768
Number of Unique Visited States: 30149
Execution Time: 4.264825344085693

***Slow A_STAR Search***
Path Cost: 25
Path: RURDDDRRDRDDRRURRDLLLLULU
Number of Visited States: 127769
Number of Unique Visited States: 88256
Execution Time: 13.292806148529053

***Weighted A_STAR Search with Alpha = 5***
Path Cost: 26
Path: RURDDDDRRDLURRDRRDRRULULDD
Number of Visited States: 382
Number of Unique Visited States: 380
Execution Time: 0.044501543045043945

***Weighted A_STAR Search with Alpha = 1.5***
Path Cos

### Weighted A*
<br/>
<div style="text-align: justify;"> 
In this version of A*, heuristic is no longer admissible because first solution found is not guaranteed to be
optimal. This algorithm trades off optimality for speed. Nodes can be expanded multiple times. Cost of solution found <i>C</i> is bounded by <i>w</i>:
$$ C \leqslant w \text{C*} $$<br>  
where <i>C*</i> is cost of the optimal solution.
    
The results show that the closer the weight is to 1, the more optimal the answer is. So with the weight close to 2, the path cost is closer to the optimal path cost. 
</div>

### Comparison between BFS and IDS
<br/>
<div style="text-align: justify;"> 
When the solution is in higher depth we should choose IDS.
But if time and memory is very critical, it is better to use BFS.
</div>

### Comparison between BFS and A*
<br/>
<div style="text-align: justify;"> 
We know that the complexity of FIFO queue is better than
the min-heap queue. So if we have an admissible heuristic, we
should choose A*. Also the time complexity of A* is better
than BFS. And it is faster when we have a higher branching
factor. But if our heuristic is not good and optimality and
memory is important for us, we should choose BFS.
</div>

### Comparison between A* and IDS
<br/>
<div style="text-align: justify;"> 
We know that uninformed search algorithms are slower than
informed search algorithms. So if we have an admissible
heuristic, we should choose A*. But if our heuristic is not good
and optimality and memory is important for us, we should
choose IDS.
</div>

### Conclusion
<br/>
<div style="text-align: justify;"> 
In this computer assignment we learned that informed
search algorithms are much faster than uninformed
algorithms. Also we were introduced one of the applications of
Artificial Intelligence in real life.
</div>